# Hamiltonian Mapper I

## Use Sympy to map objective functions into Hamiltonians consisting of pauli matrices.

In [1]:
# Import Sympy and all it libraries
from sympy import *

In [5]:
X,Y,Z,I= symbols('X Y Z I')

variable_eqn = 0.5*(I - Z)

In [6]:

MAX_CUT = 0.5 * (1 - X1*X2)

NameError: name 'X1' is not defined